In [411]:
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta
import datetime
import os

SPREADSHEET_IDS = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "コンサートホールエフ成増": "1EDY2RfjDQNsapVrl2X-UrqPKoXrkQmYJnk3uPqccBxY",
    "第一プラザ坂戸1000": "170MVr-BB3LG-g5ItkDT-8TE6R68RW9zJhRfpvQiy-PE",
    "第一プラザみずほ台店": "1_1722pigi_Z1D6eH0tsPfMneGoS9O09fyqD6F-h1mQA",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
    "ニュークラウン川越2号店": "1jocaH94-5GKWUqEqcrybgFcpb1Tn0BhvLMqaADppNWg",
    "TOHO川越店": "1VOr7BNMB-xnHihZrk6MFmetlZ0nCFP86YcrJu9w0dac",
}
# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)

In [412]:
def create_df_from_database(hall_name, start_date, end_date, model_name=None):
    # Table name 取得
    DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # print(tables)

    cursor.execute(
        "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + hall_name + "%",)
    )
    results = cursor.fetchall()

    # 結果表示
    if results:
        hall_id, hall_name = results[0]
        print(f"🔍 '{hall_name}' を含むホール名が見つかりました。")
    else:
        print(f"❌ '{hall_name}' を含むホール名は見つかりませんでした。")

    query = """
        -- 出玉データにホール名と機種名を結合して取得
        SELECT
            r.*, 
            h.name AS hall_name,
            m.name AS model_name
        FROM results r
        JOIN halls h ON r.hall_id = h.hall_id
        JOIN models m ON r.model_id = m.model_id
        WHERE h.name = ?
        AND r.date BETWEEN ? AND ?
        """

    params = [hall_name, start_date, end_date]
    if model_name:
        query += " AND m.name LIKE ?"
        params.append(f"%{model_name}%")  # 部分一致にする

    query += " ORDER BY r.date DESC, r.unit_no ASC"

    df = pd.read_sql_query(query, conn, params=params)
    print(f"データサイズ: {df.shape[0]} x {df.shape[1]}")
    print(f"📅 検索期間: {start_date} ～ {end_date}", f"📅 抽出期間: {df.date.min()} ～ {df.date.max()}")
    print(f'含まれる日数 : {df["date"].nunique()}')

    return df

In [413]:
# ブドウシミュレーター
def grape_calc_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (
        -medals
        - (
            game * 3
            - (
                bb * bb_medals
                + rb * rb_medals
                + game * replay_rate
                + game * cherry_rate_high
            )
        )
    ) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    return grape_rate


GRAPE_CONSTANTS = {
    "マイジャグラーV": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.05847,
        "cherryOn": 0.04228,
    },
    "アイムジャグラーEX-TP": {
        "bb": 251.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.06068,
        "cherryOn": 0.040475,
    },
    "ゴーゴージャグラー3": {
        "bb": 239.00,
        "rb": 95.00,
        "replay": 0.411,
        "cherryOff": 0.0661,
        "cherryOn": 0.0372,
    },
    "ファンキージャグラー2": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.0603,
        "cherryOn": 0.04324,
    },
}


def calc_grape_rate(row, constants, cherry=True):
    model = row["model_name"]
    if model not in constants:
        return None
    try:
        game = row["game"]
        bb = row["BB"]
        rb = row["RB"]
        medals = row["medals"]
        # 定数取得
        c = constants[model]
        cherry_rate = c["cherryOn"] if cherry else c["cherryOff"]
        # 分母計算式
        denominator = (
            -medals
            - (
                game * 3
                - (
                    bb * c["bb"]
                    + rb * c["rb"]
                    + game * c["replay"]
                    + game * cherry_rate
                )
            )
        ) / 8
        if denominator == 0:
            return None  # ゼロ除算防止
        grape = (game / denominator) - ((game / denominator) * 2)
        return round(grape, 2)

    except Exception as e:
        print(f"⚠️ Grape計算失敗: {model} → {e}")
        return None


def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"


def df_preprocessing(df, hall_name):
    json_path = f"C:/python/dataOnline/anaslo_02/json/{hall_name}_area_map.json"
    if not os.path.exists(json_path):
        json_path = f"C:/python/dataOnline/anaslo_02/json/other_area_map.json"
    print(f"データ前処理を行います")
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre_columns = ["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]
    df_pre = df_pre[df_pre_columns]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    df_pre["Grape_rate"] = grape_calc_myfive(
        df_pre["game"], df_pre["BB"], df_pre["RB"], df_pre["medals"], cherry=True).round(2)
    
    df_pre["Grape_rate"] = df_pre.apply(lambda row: calc_grape_rate(row, GRAPE_CONSTANTS), axis=1)
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)
    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["year"] = df_pre["date"].dt.year
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]

    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, json_path))
    
    df_pre.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())
    # for i, model in enumerate(model_list):
    #     print(f"{i+1}: {model}", end=", ")

    return df_pre, model_list


def add_spreadsheet(spreadsheet, sheet_name, df, sheet_clear=False):
    today = datetime.datetime.today()
    rows, cols = df.shape
    try:
        sheet = spreadsheet.worksheet(sheet_name)
        print(f"✅ シート「{sheet_name}」が既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(
            title=sheet_name, rows=str(rows + 3), cols=str(cols + 3)
        )
        print(f"🆕 シート「{sheet_name}」を新規作成しました。")
    if sheet_clear:
        sheet.clear()
    last_row = len(sheet.get_all_values())
    set_with_dataframe(sheet, df, row=last_row, include_index=True)
    sheet.update_cell(1, 1, today.strftime("%Y-%m-%d UPDATED"))
    print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

## ピボットテーブル作成


In [414]:
def create_pivot_table(
    df,
    index,
    columns,
    pivots=["game", "medals", "BB", "RB"],
    reverse=False,
    margins=True,
    day_target=None,
):
    df_filtered = df.copy()
    if day_target is not None:
        df_filtered = df_filtered[df_filtered["day"] == day_target]

    pivot_results = {}
    for col in pivots:
        table = df_filtered.pivot_table(
            index=index,
            columns=columns,
            values=col,
            aggfunc="sum",
            margins=margins,
            margins_name="total",
        )
        pivot_results[col] = table
        if reverse:
            pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]
    merged = pd.concat([game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]
    merged.replace([np.inf, -np.inf, np.nan], None, inplace=True)
    details = {
        "game": game,
        "medals": medals,
        "medal_rate": medal_rate,
        "bb": bb,
        "rb": rb,
        "rb_rate": rb_rate,
        "total_rate": total_rate,
    }

    return merged, details



## 出力データ

- 期間指定
  - 機種別 x 日付
  - 島別 x 日付
  - 台番号 x 日付
  - 島別 x 月
  - 過去 n 日の差枚比較

In [430]:
hall_name = "EXA FIRST"
# hall_name = "コンサートホールエフ成増"
hall_name = "第一プラザみずほ台店"
# hall_name = "オータ志木駅前店"
# hall_name = "第一プラザ坂戸1000"
# hall_name = "ニュークラウン川越2号店"
# hall_name = "TOHO川越店"
# hall_name = "グランドオータ新座駅前店"

model_name = "ジャグラー"

today = datetime.date.today()
start_date = today - relativedelta(months=6, days=today.day - 1)
df_db = create_df_from_database(hall_name, start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)
spreadsheet = client.open_by_key(SPREADSHEET_IDS[hall_name])
print(f"スプレッドシート: {spreadsheet.title} を開きました。")
df.tail()

🔍 '第一プラザみずほ台店' を含むホール名が見つかりました。
データサイズ: 10151 x 11
📅 検索期間: 2024-11-01 ～ 2025-05-25 📅 抽出期間: 2024-11-01 ～ 2025-05-24
含まれる日数 : 185
データ前処理を行います
スプレッドシート: 第一プラザみずほ台店 を開きました。


,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,year,unit_last,area
10146,第一プラザみずほ台店,2024-11-01,マイジャグラーV,520,6217,20,17,-686,310.8,365.7,5.69,168.0,2024-11,1,4,2024,0,その他
10147,第一プラザみずほ台店,2024-11-01,マイジャグラーV,521,6862,33,17,1837,207.9,403.6,5.60,137.2,2024-11,1,4,2024,1,その他
10148,第一プラザみずほ台店,2024-11-01,マイジャグラーV,522,6313,23,12,-680,274.5,526.1,5.77,180.4,2024-11,1,4,2024,2,その他
10149,第一プラザみずほ台店,2024-11-01,マイジャグラーV,523,7825,36,32,2953,217.4,244.5,5.58,115.1,2024-11,1,4,2024,3,その他
10150,第一プラザみずほ台店,2024-11-01,マイジャグラーV,524,7539,32,38,2581,235.6,198.4,5.74,107.7,2024-11,1,4,2024,4,その他


## MODEL_RATE


In [431]:

sheet_name = "MODEL_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
model_rate = details["medal_rate"].copy()
model_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, model_rate, sheet_clear=True)
model_rate.head()

MEDAL_RATE                                                   \
day                    1      2      3      4      5      6      7      8   
model_name                                                                  
アイムジャグラーEX-TP      0.974  0.976  0.988  1.012  0.994  0.974  1.001  0.984   
ウルトラミラクルジャグラー      0.988  0.945  1.056  0.996  0.982  0.953  0.962  1.010   
ゴーゴージャグラー3         0.966  0.997  0.957  1.002  1.016  1.007  0.988  0.999   
ジャグラーガールズ          0.959  0.980  1.006  0.963  0.963  0.986  0.941  0.980   
ハッピージャグラーVIII      0.981  0.962  0.983  0.969  0.974  0.988  0.971  0.927   

                             ...                                            \
day                9     10  ...     23     24     25     26     27     28   
model_name                   ...                                             
アイムジャグラーEX-TP  0.985  0.995  ...  0.995  0.985  0.981  0.970  0.979  0.974   
ウルトラミラクルジャグラー  0.952  0.990  ...  0.977  0.976  1.003  0.980  0.923  1.009   
ゴーゴージャグラー3     0.942  0.980  ...  0.980  1.017  0.996  0.989  0.950  0.975   
ジャグラーガールズ      0.945  0.963  ...  1.012  0.941  0.966  0.991  1.039  0.966   
ハッピージャグラーVIII  0.936  0.945  ...  0.960  0.933  0.981  0.995  0.967  0.942   

                                           
day               29     30     31  total  
model_name                                 
アイムジャグラーEX-TP  0.982  0.975  0.962  0.990  
ウルトラミラクルジャグラー  0.983  1.015  0.988  0.981  
ゴーゴージャグラー3     0.957  0.995  0.980  0.990  
ジャグラーガールズ      1.002  0.892  0.928  0.976  
ハッピージャグラーVIII  0.957  0.996  0.972  0.965  

[5 rows x 32 columns]

## ISLAND_RATE


In [432]:
sheet_name = f"ISLAND_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["area"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
island_rate = details["medal_rate"].copy()
island_rate.to_csv(csv_path)
add_spreadsheet(spreadsheet, sheet_name, island_rate, sheet_clear=True)
island_rate.head()

✅ シート「ISLAND_RATE」が既に存在します。
✅ シート 'ISLAND_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                               \
day            1      2      3      4    5      6      7      8      9     10   
area                                                                            
その他         0.99  0.985  0.994  0.998  1.0  0.992  0.992  0.989  0.982  0.989   
total       0.99  0.985  0.994  0.998  1.0  0.992  0.992  0.989  0.982  0.989   

       ...                                                                 \
day    ...     23     24     25     26     27     28     29     30     31   
area   ...                                                                  
その他    ...  0.998  0.989  0.996  0.987  0.987  0.979  0.978  0.982  0.986   
total  ...  0.998  0.989  0.996  0.987  0.987  0.979  0.978  0.982  0.986   

              
day    total  
area          
その他    0.992  
total  0.992  

[2 rows x 32 columns]

## UNIT_RATE


In [433]:
sheet_name = f"UNIT_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name", "unit_no"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
target_rate = 1.05
unit_rate = details["medal_rate"].copy()
unit_rate[("MEDAL_RATE", f"{target_rate}+")] = (
    unit_rate.iloc[:, :-1] >= target_rate
).sum(axis=1)
countif = (unit_rate.iloc[:-1, :] >= target_rate).sum(axis=0)
unit_rate = pd.concat(
    [unit_rate, pd.DataFrame([countif], index=[(f"count_{target_rate}+", "")])], axis=0
)
unit_rate.replace([np.inf, -np.inf], np.nan, inplace=True)
unit_rate = unit_rate[~unit_rate.iloc[:, 1].isna()]
unit_rate.to_csv(csv_path)
add_spreadsheet(spreadsheet, sheet_name, unit_rate, sheet_clear=True)
unit_rate.head()

✅ シート「UNIT_RATE」が既に存在します。
✅ シート 'UNIT_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                   \
day                        1      2      3      4      5      6      7      8   
アイムジャグラーEX-TP 420      0.979  0.941  1.068  1.056  0.967  0.982  1.017  0.992   
              421      0.958  0.988  1.073  0.967  0.990  0.990  1.091  1.062   
              422      0.970  0.933  1.013  0.976  1.030  0.967  0.907  0.920   
              423      0.964  0.900  0.964  1.100  0.975  0.859  0.999  1.020   
              424      0.913  0.997  0.907  0.952  0.988  0.999  1.078  1.022   

                                 ...                                     \
day                    9     10  ...     24     25     26     27     28   
アイムジャグラーEX-TP 420  0.978  0.961  ...  0.990  0.979  0.955  0.973  0.916   
              421  0.925  0.983  ...  0.860  0.986  1.047  0.973  0.945   
              422  1.041  0.933  ...  0.942  1.039  1.020  1.011  0.989   
              423  0.978  0.980  ...  1.023  0.948  0.945  0.958  0.875   
              424  0.960  0.958  ...  1.011  1.006  0.941  0.985  1.034   

                                                     
day                   29     30     31  total 1.05+  
アイムジャグラーEX-TP 420  0.990  0.992  0.948  0.988     2  
              421  0.894  1.008  1.019  0.990     4  
              422  1.028  0.961  0.851  1.001     4  
              423  0.909  0.979  0.968  0.984     2  
              424  1.031  0.922  0.717  0.988     1  

[5 rows x 33 columns]

# DAYn

- SHEET_NAME : DAY1，DAY2，… DAY31


In [434]:
index = ["area", "unit_no"]
columns = ["date"]
day_list = [25]
# for day_target in range(today.day - 1, today.day + 1):
for day_target in day_list:
    sheet_name = f"DAY{day_target}"
    csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
    merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True, day_target=day_target)
    merged.to_csv(csv_path)
    add_spreadsheet(spreadsheet, f"DAY{day_target}", merged, sheet_clear=True,)
merged.head()

✅ シート「DAY25」が既に存在します。
✅ シート 'DAY25' に DataFrame を書き込みました！


GAME MEDALS    BB    RB MEDAL_RATE RB_RATE TOTAL_RATE  \
date          total  total total total      total   total      total   
area unit_no                                                           
その他  408      29150    818   103    96      1.009   303.6      146.5   
     409      26004   5093   118    78      1.065   333.4      132.7   
     410      27092   1263   110    79      1.016   342.9      143.3   
     411      12076  -2638    41    22      0.927   548.9      191.7   
     412      25074    601    96    73      1.008   343.5      148.4   

                            GAME              MEDALS                  BB  ...  \
date         2025-04-25 00:00:00 2025-04-25 00:00:00 2025-04-25 00:00:00  ...   
area unit_no                                                              ...   
その他  408                  8225.0               541.0                29.0  ...   
     409                  8388.0              4081.0                48.0  ...   
     410                  5462.0              -564.0                21.0  ...   
     411                  3493.0              -750.0                12.0  ...   
     412                  3476.0               785.0                16.0  ...   

                      MEDAL_RATE             RB_RATE          TOTAL_RATE  \
date         2024-12-25 00:00:00 2024-12-25 00:00:00 2024-12-25 00:00:00   
area unit_no                                                               
その他  408                   0.833               375.3               218.9   
     409                   1.004               248.5               133.8   
     410                   1.027               261.4               130.7   
     411                   0.775               349.5               233.0   
     412                   0.863               414.2               207.1   

                            GAME              MEDALS                  BB  \
date         2024-11-25 00:00:00 2024-11-25 00:00:00 2024-11-25 00:00:00   
area unit_no                                                               
その他  408                  2083.0             -1340.0                 3.0   
     409                  4491.0              1644.0                25.0   
     410                  2448.0               484.0                12.0   
     411                   718.0              -797.0                 1.0   
     412                  4033.0              -173.0                15.0   

                              RB          MEDAL_RATE             RB_RATE  \
date         2024-11-25 00:00:00 2024-11-25 00:00:00 2024-11-25 00:00:00   
area unit_no                                                               
その他  408                     5.0               0.786               416.6   
     409                    11.0               1.122               408.3   
     410                     6.0               1.066               408.0   
     411                     0.0                0.63                None   
     412                     9.0               0.986               448.1   

                      TOTAL_RATE  
date         2024-11-25 00:00:00  
area unit_no                      
その他  408                   260.4  
     409                   124.8  
     410                   136.0  
     411                   718.0  
     412                   168.0  

[5 rows x 42 columns]

## MONTH_RATE

In [429]:
start_date = today - relativedelta(months=100, days=today.day-1)
df_db = create_df_from_database("EXA FIRST", start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)
sheet_name = "MONTH_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["month"]
merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True)
month_rate = details["medal_rate"].copy()
add_spreadsheet(spreadsheet, sheet_name, month_rate, sheet_clear=True)
month_rate.to_csv(csv_path)
month_rate

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 63384 x 11
📅 検索期間: 2017-01-01 ～ 2025-05-25 📅 抽出期間: 2024-04-01 ～ 2025-05-24
含まれる日数 : 417
データ前処理を行います
🆕 シート「MONTH_RATE」を新規作成しました。
✅ シート 'MONTH_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                  \
month              total 2025-05 2025-04 2025-03 2025-02 2025-01 2024-12   
model_name                                                                 
アイムジャグラーEX-TP      1.002   0.996   0.998   1.001   0.993   0.997   1.000   
ウルトラミラクルジャグラー      0.998   0.988   0.993   0.997   1.002   1.000   1.001   
ゴーゴージャグラー3         1.002   1.000   0.993   0.992   1.001   1.000   0.996   
ジャグラーガールズ          1.003   0.999   0.993   1.001   0.990   0.996   0.994   
ハッピージャグラーVIII      0.995   0.992   0.995     NaN     NaN     NaN     NaN   
ファンキージャグラー2        0.996   0.991   0.992   0.993   0.996   0.990   0.995   
マイジャグラーV           1.010   1.003   1.000   1.002   1.003   1.006   1.005   
ミスタージャグラー          1.001   0.998   1.002   0.991   0.996   0.990   1.006   
total              1.006   0.999   0.998   0.999   1.001   1.002   1.002   

                                                                               
month         2024-11 2024-10 2024-09 2024-08 2024-07 2024-06 2024-05 2024-04  
model_name                                                                     
アイムジャグラーEX-TP   1.001   0.999   1.001   1.000   1.007   1.005   1.004   1.008  
ウルトラミラクルジャグラー     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN  
ゴーゴージャグラー3      1.005   0.995   0.998   1.003   1.003   1.006   1.010   1.012  
ジャグラーガールズ       0.995   0.999   1.003   0.997   1.005   1.005   1.010   1.016  
ハッピージャグラーVIII   0.957   1.004     NaN     NaN     NaN     NaN     NaN     NaN  
ファンキージャグラー2     0.996   1.000   0.989   1.001   0.986   0.990   1.000   1.007  
マイジャグラーV        1.002   1.008   1.009   1.012   1.011   1.017   1.021   1.025  
ミスタージャグラー       0.997   1.002   1.008   1.007   1.009     NaN     NaN     NaN  
total           1.001   1.004   1.005   1.007   1.008   1.011   1.015   1.018

## HISTORY

In [428]:
start_date = datetime.date(today.year, today.month-1, 1) - relativedelta(days=6)
df_db = create_df_from_database(hall_name, start_date, today, model_name="ジャグラー")
df, model_list = df_preprocessing(df_db, hall_name)

pivot_targets = ["medals", "game", "RB_rate", "Total_rate", "Grape_rate"]
index_targets = ["area", "model_name", "unit_no"]
columns_targets = ["date"]

merged_by_model = pd.DataFrame()
for model in df["model_name"].unique():
    
    # モデルごとにデータをフィルタリング
    df_model = df[df["model_name"] == model]
    pivot_results = {}
    for col in pivot_targets:
        table = df_model.pivot_table(
            index=index_targets,
            columns=columns_targets,
            values=col,
            aggfunc="sum",
        )
        pivot_results[col] = table
    
    medals = pivot_results["medals"]
    game = pivot_results["game"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]
    grape_rate = pivot_results["Grape_rate"]
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)
    
    # ランキング列作成
    rolling7 = medals.T.rolling(7, min_periods=7).sum().T
    medal_rank7 = rolling7.rank(method="min", ascending=True)
    medal_rank7 = medal_rank7.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    rolling5 = medals.T.rolling(5, min_periods=5).sum().T
    medal_rank5 = rolling5.rank(method="min", ascending=True)
    medal_rank5 = medal_rank5.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    rolling3 = medals.T.rolling(3, min_periods=3).sum().T
    medal_rank3 = rolling3.rank(method="min", ascending=True)
    medal_rank3 = medal_rank3.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    medal_rank1 = medals.rank(method="min", ascending=True)
    medal_rank1 = medal_rank1.fillna(0).replace([np.inf, -np.inf], 0).astype(int)

    # MultiIndex化（ラベル付け）
    labeled_tables = [
        ("GRAPE_RATE", grape_rate),
        ("TOTAL_RATE", total_rate),
        ("RB_RATE", rb_rate),
        ("GAME", game),
        ("RATE_MEDAL", medal_rate),
        ("MEDALS", medals),
        ("3ROLLING", rolling3),
        ("5ROLLING", rolling5),
        ("7ROLLING", rolling7),
        ("1RANK", medal_rank1),
        ("3RANK", medal_rank3),
        ("5RANK", medal_rank5),
        ("7RANK", medal_rank7),
    ]
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去・日付ソート・一部データ削除
    interleaved_cols = [
        col
        for col_group in zip(*(df.columns for _, df in labeled_tables))
        for col in col_group
    ]
    merged = pd.concat([df for _, df in labeled_tables], axis=1)[interleaved_cols]
    merged = merged.iloc[:, ::-1]
    merged = merged[~merged.iloc[:, 5].isna()]

    # エリアごとに空行挿入して整形
    merged_by_area = pd.DataFrame()
    for area in merged.index.get_level_values("area").unique():
        area_merged = merged.xs(area, level="area", drop_level=False)
        if not area_merged.empty:
            empty_index = pd.MultiIndex.from_tuples(
                [("", " ", " ")], names=merged.index.names
            )
            empty_row = pd.DataFrame(
                [[""] * area_merged.shape[1]],
                index=empty_index,
                columns=area_merged.columns,
            )
            merged_by_area = pd.concat([merged_by_area, area_merged, empty_row])

    # インデックス削除
    merged_by_area = merged_by_area.droplevel("area")
    
    if not merged_by_area.empty:
        empty_index = pd.MultiIndex.from_tuples(
            [(" ", " ")], names=merged_by_area.index.names
        )
        empty_row = pd.DataFrame(
            [[""] * merged_by_area.shape[1]],
            index=empty_index,
            columns=merged_by_area.columns,
        )
        merged_by_model = pd.concat(
            [merged_by_model, merged_by_area, empty_row], axis=0
        )

# 累計した最後の7日を削除
merged_by_model = merged_by_model.iloc[:, :-len(labeled_tables)*6]
merged_by_model.replace([np.inf, -np.inf], np.nan, inplace=True)

csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
merged_by_model.to_csv(csv_path)

sheet_name = "HISTORY"
add_spreadsheet(spreadsheet, sheet_name, merged_by_model, sheet_clear=True)

merged_by_model.head()

🔍 '第一プラザみずほ台店' を含むホール名が見つかりました。
データサイズ: 2707 x 11
📅 検索期間: 2025-03-26 ～ 2025-05-25 📅 抽出期間: 2025-03-27 ～ 2025-05-24
含まれる日数 : 49
データ前処理を行います
✅ シート「HISTORY」が既に存在します。
✅ シート 'HISTORY' に DataFrame を書き込みました！


7RANK      5RANK      3RANK      1RANK   7ROLLING  \
date               2025-05-24 2025-05-24 2025-05-24 2025-05-24 2025-05-24   
model_name unit_no                                                          
マイジャグラーV   408              9          3          1          6     -250.0   
           409             20         20          6         11     4174.0   
           410              3          2          7          7    -2163.0   
           411              5          5         10         15    -1425.0   
           412              4          7         14         13    -1686.0   

                     5ROLLING   3ROLLING     MEDALS RATE_MEDAL       GAME  \
date               2025-05-24 2025-05-24 2025-05-24 2025-05-24 2025-05-24   
model_name unit_no                                                          
マイジャグラーV   408        -2209.0    -2720.0       -738      0.957       5685   
           409         5302.0    -1454.0       -227      0.955       1688   
           410        -2646.0    -1239.0       -669      0.957       5196   
           411        -1576.0     -773.0        802      1.032       8260   
           412        -1151.0      448.0       -134      0.973       1679   

                    ...      1RANK   7ROLLING   5ROLLING   3ROLLING  \
date                ... 2025-04-08 2025-04-08 2025-04-08 2025-04-08   
model_name unit_no  ...                                               
マイジャグラーV   408      ...          2     5186.0     3500.0      843.0   
           409      ...          1    -2122.0    -1035.0     1727.0   
           410      ...         10     3721.0     2913.0     2337.0   
           411      ...         17    -2349.0    -1395.0      296.0   
           412      ...         14     2749.0     2912.0     3523.0   

                       MEDALS RATE_MEDAL       GAME    RB_RATE TOTAL_RATE  \
date               2025-04-08 2025-04-08 2025-04-08 2025-04-08 2025-04-08   
model_name unit_no                                                          
マイジャグラーV   408           -622      0.845       1341      670.5      268.2   
           409           -808      0.634        736        0.0        0.0   
           410            -99      0.925        442        0.0      221.0   
           411            826      1.084       3263      543.8      141.9   
           412             64      1.025        861      287.0      143.5   

                   GRAPE_RATE  
date               2025-04-08  
model_name unit_no             
マイジャグラーV   408           5.69  
           409           5.52  
           410           6.45  
           411            5.8  
           412            5.5  

[5 rows x 559 columns]